In [1]:
import cv2 as cv
import numpy as np

In [2]:
def reframe(frame, scale=0.75):
    width = int(frame.shape[1] * scale)
    height = int(frame.shape[0] * scale)
    dim = (width, height)
    return cv.resize(frame, dim, interpolation=cv.INTER_AREA)

In [ ]:
capture = cv.VideoCapture('URL')
capture2 = cv.VideoCapture('URL')

while True:
    ret, frame = capture.read()
    ret2, frame2 = capture2.read()
    if not ret or not ret2:
        print("Failed to capture video from one of the cameras.")
        break

    
    # Calculate absolute difference
    diff = cv.absdiff(frame, frame2)

    gray = cv.cvtColor(diff, cv.COLOR_BGR2GRAY)  # Convert to grayscale

    # Threshold the difference to highlight significant changesnificant changes
    _, thresh = cv.threshold(gray, 125, 255, cv.THRESH_BINARY)

    # # Dilate to make the changes more visible
    # kernel = np.ones((5, 5), np.uint8)
    # dilated = cv.dilate(thresh, kernel, iterations=2)

    # Find contours in the difference
    contours, _ = cv.findContours(thresh, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    # Draw red boxes where changes occurred
    result = frame2.copy()
    for contour in contours:
        if cv.contourArea(contour) > 500:  # Ignore tiny changes
            (x, y, w, h) = cv.boundingRect(contour)
            cv.rectangle(result, (x, y), (x + w, y + h), (0, 0, 255), 2)


    # Display the frame
    cv.imshow('Video', frame)
    cv.imshow('Video2', frame2)
    cv.imshow('Difference', result)


    # Break the loop if 'q' is pressed
    if cv.waitKey(1) & 0xFF == ord('q'):
        capture.release()
        capture2.release()
        cv.destroyAllWindows()  # Calculate absolute difference
    


In [22]:
# Playing with the contours

capture3 = cv.VideoCapture('URL')

while True:
    ret3, frame3 = capture3.read()
    if not ret3:
        print("Failed to capture video from the camera.")
        break

    frame3 = reframe(frame3, 0.5)
    
    # Convert to grayscale and blur
    gray3 = cv.cvtColor(frame3, cv.COLOR_BGR2GRAY)
    gray3 = cv.GaussianBlur(gray3, (5, 5), cv.BORDER_DEFAULT)

    # Threshold
    _, thresh = cv.threshold(gray3, 125, 255, cv.THRESH_BINARY)

    # Find contours from the thresholded (binary) image
    contours3, _ = cv.findContours(thresh, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)

    # Blank frame in color (for drawing red contours)
    blank_frame = np.zeros_like(frame3)

    # Draw contours in red on the blank frame
    result3 = cv.drawContours(blank_frame.copy(), contours3, -1, (0, 0, 255), 2)

    # Canny edge detection
    canny = cv.Canny(thresh, 125, 175)

    # Show everything
    cv.imshow('Original', frame3)
    cv.imshow('Gray', gray3)
    cv.imshow('Thresh', thresh)
    cv.imshow('Canny Edges', canny)
    cv.imshow('Blank with Contours', result3)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break

capture3.release()
cv.destroyAllWindows()


    

In [36]:
## Color differences instead of absolute differences between two frames

capture2 = cv.VideoCapture('URL')
capture3 = cv.VideoCapture('URL')

while True:
    ret2, frame2 = capture2.read()
    ret3, frame3 = capture3.read()
    if not ret2 or not ret3:
        print("Failed to capture video from one of the cameras.")
        break
    target_size = (640, 480)  # Or whatever size you want

    frame2 = cv.resize(frame2, target_size)
    frame3 = cv.resize(frame3, target_size)

    # Calculate color differences
    b1,g1,r1 = cv.split(frame2)
    b2,g2,r2 = cv.split(frame3)

    diff_b = cv.absdiff(b1, b2)
    diff_g = cv.absdiff(g1, g2)
    diff_r = cv.absdiff(r1, r2)

    diff_b_colored = cv.cvtColor(diff_b, cv.COLOR_GRAY2BGR)
    diff_g_colored = cv.cvtColor(diff_g, cv.COLOR_GRAY2BGR)
    diff_r_colored = cv.cvtColor(diff_r, cv.COLOR_GRAY2BGR)

    # Add text labels
    cv.putText(diff_b_colored, "Blue", (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    cv.putText(diff_g_colored, "Green", (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv.putText(diff_r_colored, "Red", (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Stack the three grayscale difference channels horizontally
    stacked = cv.hconcat([diff_b_colored, diff_g_colored, diff_r_colored])

    # Display the frames and the difference
    cv.imshow('Frame 2', frame2)
    cv.imshow('Frame 3', frame3)
    cv.imshow('Color Difference', stacked)

    if cv.waitKey(1) & 0xFF == ord('q'):
        break

capture2.release()
capture3.release()
cv.destroyAllWindows()

